In [4]:
import numpy as np

# translation matrix
trans = np.array(
    [[1, 0, 0, 1],
     [0, 1, 0, 2],
     [0, 0, 1, 3],
     [0, 0, 0, 1]]
)

# rotation matrix
rot = np.array(
    [[ 0.4198, -0.3450, -0.8395, 0.0000],
     [-0.2159,  0.8605, -0.4615, 0.0000],
     [ 0.8816,  0.3749,  0.2867, 0.0000],
     [ 0.0000,  0.0000,  0.0000, 1.0000]]
)

# rototranslation matrix
tr = trans @ rot
rt = rot @ trans
print(tr)
print(rt)

[[ 0.4198 -0.345  -0.8395  1.    ]
 [-0.2159  0.8605 -0.4615  2.    ]
 [ 0.8816  0.3749  0.2867  3.    ]
 [ 0.      0.      0.      1.    ]]
[[ 0.4198 -0.345  -0.8395 -2.7887]
 [-0.2159  0.8605 -0.4615  0.1206]
 [ 0.8816  0.3749  0.2867  2.4915]
 [ 0.      0.      0.      1.    ]]


Matrix TR is trivial to construct.

In [16]:
# Effect on an object
obj = np.array(
    [[1, 2, 3, 1],
     [4, 5, 6, 1],
     [7, 8, 9, 1],
     [10, 11, 12, 1]]
)

obj_t = trans @ obj
obj_t_r = rot @ obj_t
print("translate then rotate", obj_t_r)

obj_r = rot @ obj
obj_r_t = trans @ obj_r
print("rotate then translate", obj_r_t)

obj_tr = tr @ obj
print("tr", obj_tr)

obj_rt = rt @ obj
print("rt", obj_rt)

translate then rotate [[-34.7237 -38.2771 -41.8305  -3.5534]
 [  1.2016   1.5053   1.809    0.3037]
 [ 29.3031  33.3378  37.3725   4.0347]
 [ 10.      11.      12.       1.    ]]
rotate then translate [[ 3.1633  3.3986  3.6339  0.2353]
 [19.9956 22.1787 24.3618  2.1831]
 [34.3881 38.9313 43.4745  4.5432]
 [10.     11.     12.      1.    ]]
tr [[ 3.1633  3.3986  3.6339  0.2353]
 [19.9956 22.1787 24.3618  2.1831]
 [34.3881 38.9313 43.4745  4.5432]
 [10.     11.     12.      1.    ]]
rt [[-34.7237 -38.2771 -41.8305  -3.5534]
 [  1.2016   1.5053   1.809    0.3037]
 [ 29.3031  33.3378  37.3725   4.0347]
 [ 10.      11.      12.       1.    ]]


Matrix TR corrensponds to first rotating the object and then translating it.

How does this work if we rotate the camera?

In [37]:
from transforms3d.rotations import rotate
import torch
from models.neural_renderer import load_model

model = load_model('trained-models/chairs.pt')

volume = torch.rand(1, 64, 32, 32, 32)

print(torch.tensor(trans, dtype=torch.float32).unsqueeze(0).shape)
print(volume.shape)

t = torch.tensor(trans, dtype=torch.float32).unsqueeze(0)
r = torch.tensor(rot, dtype=torch.float32).unsqueeze(0)
rt = torch.tensor(rt, dtype=torch.float32).unsqueeze(0)
tr = torch.tensor(tr, dtype=torch.float32).unsqueeze(0)

volume_t = model.rotate(volume, t)
volume_t_r = model.rotate(volume_t, r)
print("translate then rotate", volume_t_r[0, 0, 0, 0, :3])

volume_r = model.rotate(volume, r)
volume_r_t = model.rotate(volume, t)
print("rotate then translate", volume_r_t[0, 0, 0, 0, :3])

volume_tr = model.rotate(volume, tr)
print("tr", volume_tr[0, 0, 0, 0, :3])

volume_rt = model.rotate(volume, rt)
print("rt", volume_rt[0, 0, 0, 0, :3])

torch.Size([1, 4, 4])
torch.Size([1, 64, 32, 32, 32])
translate then rotate tensor([0.0000, 0.2324, 0.1720])
rotate then translate tensor([0.7657, 0.9083, 0.7558])
tr tensor([0.0000, 0.2324, 0.1720])
rt tensor([0.0000, 0.2324, 0.1720])
